In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import skew, kurtosis, shapiro
plt.style.use(['bmh'])
import talib
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()
import yaml
from utils import utils, telebot

In [ ]:
all_config = yaml.safe_load(open("../config_file.yml", "r"))

In [ ]:
df_input = utils.get_ticks_data('../src/data/example_data.csv')

In [ ]:
start_date, end_date = utils.get_start_end(all_config)
source = all_config['stocks']['source']
metric = all_config['stocks']['metric']
ticks = df_input['TICK'].to_list()

In [ ]:
df = utils.get_stock_data(ticks, start_date, end_date, col=metric)

In [ ]:
print("Following ticks will be analyzed: \n %s" % df.columns)

In [ ]:
df_clean = utils.get_metrics(df, all_config['model']['kpis'])

In [ ]:
df_clean['rsi_f'] = np.where(df_clean['rsi'] <= 45, 1, 0)
df_clean['bb_f'] = np.where(df_clean['price'] <= df_clean['bb_low'], 1, 0)
df_clean['ema_f'] = np.where(df_clean['ema_low'] >= df_clean['ema_up'], 1, 0)
df_clean['score'] = df_clean['ema_f'] + df_clean['rsi_f'] + df_clean['bb_f']

In [ ]:
date_max = (max(df_clean.index)).strftime("%Y-%m-%d")
dflags = df_clean[(df_clean['score']>=2) & (df_clean.index==date_max)].sort_values('score', ascending=False)

In [ ]:
dflags

In [ ]:
test = df_clean[df_clean['stock']=='AAPL']

In [ ]:
talib.EMA(test['price'], 26)

### Pending

In [ ]:
df_test = df['AAPL']

In [ ]:
talib.RSI(df[]).plot(figsize=(20,5))

In [ ]:
df[['APPL']].plot(figsize=(20,8))

In [ ]:
upper_1sd, mid_1sd, lower_1sd = talib.BBANDS(df[tk], 
                                     nbdevup=1,
                                     nbdevdn=1,
                                     timeperiod=20)
# Plot the upper and lower Bollinger Bands 
plt.figure(figsize=(20,8))
plt.plot(df[tk], color='green', label='Price')
plt.plot(upper_1sd, color='tomato', label="Upper 1sd")
plt.plot(lower_1sd, color='tomato', label='Lower 2sd')

# Customize and show the plot
plt.legend(loc='upper left')
plt.title('Bollinger Bands (1sd)')
plt.show()


# Telegram
1. Create a bot, a Telegram chat group and add the bot on it
2. Add the bot by using /my_id @<bot name> 
3. get the chat id

In [ ]:
token = all_config['bot']['chat_token']
chat_id = all_config['bot']['chat_id']

In [ ]:
bot = telebot.Tabot(token)

In [ ]:
print(type(bot))

In [ ]:
bot.chat_id

In [ ]:
bot.get_chat_id()

In [ ]:
bot.check_bot()

### Sending the msg

In [ ]:
total_ticks = len(df.columns)
msg_ticks = len(dflags.stock)

In [ ]:
msg_title = "Stocks KPIs for {end_date}: \nWe got {msg_ticks} possible ticks from {total_ticks} \n".format(end_date=end_date, msg_ticks=msg_ticks,total_ticks=total_ticks)
msg_header = '\nTICK \t PRICE RSI RSI_F BB_F EMA_F FLAGS\n'

In [ ]:
msgs = []
for row in dflags.itertuples():
    msgs.append('{s}   \t {p}\t {rsi} {rsi_f} {bb_f} {ema_f} {score} \n'.\
                format(s=row.stock, p=np.round(row.price,1), rsi=np.round(row.rsi, 1), rsi_f=row.rsi_f,bb_f= row.bb_f, ema_f=row.ema_f, score= row.score))

msg_formated = ' '.join(msgs)

In [ ]:
message = msg_title + msg_header + msg_formated

In [ ]:
bot.send_message(message)